In [1]:
#%% 0) Environment + versions (run first)
"""
Tetrode sorting pipeline (Open Ephys → SI 0.103 → KS4 & SC2 → Phy)
- Robust to multi-stream OE sessions (lets you choose stream)
- Real device-ID tetrode grouping (auto by 4, or manual list)
- Optional 2×2 tetrode geometry (for nicer Phy plots)
- Preprocessing: bandpass + 50/100/150 Hz notch + per‑tetrode CAR
- Timestamped caches to avoid Windows file-lock issues
- KS4 (CPU by default) and SC2 run in separate, independent blocks
- SortingAnalyzer sequence for SI 0.103 (random_spikes → waveforms → templates → amplitudes → PCs)
- Phy export per sorter
"""

from __future__ import annotations
from pathlib import Path
import time
import shutil
import sys
import numpy as np

import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss

# ----------------------
# Quick sanity prints
# ----------------------
try:
    import torch
    cuda_ok = torch.cuda.is_available()
except Exception:
    cuda_ok = False

print("SpikeInterface:", si.__version__)
print("Installed sorters:", ss.installed_sorters())
print("CUDA available:", cuda_ok)

SpikeInterface: 0.103.0
Installed sorters: ['kilosort4', 'simple', 'spykingcircus2', 'tridesclous2']
CUDA available: False


In [2]:
#%% 1) User settings (edit here)
# ⚙️ Run this in your **spikeinterface** conda env.

from pathlib import Path

# --- Input recording location ---
ROOT_DIR = Path(r"C:/Users/ryoi/Documents/SpikeSorting/recordings")
SESSION_SUBPATH = Path(r"2025-10-01_15-53-19/Record Node 125/experiment1/recording1")
DATA_PATH = ROOT_DIR / SESSION_SUBPATH

# If your session has multiple streams, set one explicitly or leave None to auto-list
STREAM_NAME: str | None = None
# e.g. "Record Node 125#Acquisition_Board-100.Rhythm Data"

# --- Outputs, one folder per sorter ---
BASE_DIR = Path(r"C:/Users/ryoi/Documents/SpikeSorting")
KS4_OUT = BASE_DIR / "ks4_outputs"
SC2_OUT = BASE_DIR / "sc2_outputs"

# Create output dirs if missing
for d in (KS4_OUT, SC2_OUT):
    d.mkdir(parents=True, exist_ok=True)

# --- Run a quick test on first N seconds (None = full dataset) ---
TEST_SECONDS: int | None = 60

# --- Known bad channels (use same ID format as recording.channel_ids) ---
BAD: list = []  # e.g. ["CH59", "CH12"]

# --- Optional geometry for nicer Phy plots ---
ATTACH_GEOMETRY = True
TETRODES_PER_ROW = 8  # grid layout for tetrodes

# --- Sorter toggles ---
RUN_KS4 = True
RUN_SC2 = True

# --- KS4 device ('cpu' or 'cuda') ---
KS4_TORCH_DEVICE = "cpu"

# --- Status ---
print("Using data folder:", DATA_PATH)
print("KS4 outputs →", KS4_OUT)
print("SC2 outputs →", SC2_OUT)
print(f"Test duration: {TEST_SECONDS if TEST_SECONDS else 'FULL recording'}")

Using data folder: C:\Users\ryoi\Documents\SpikeSorting\recordings\2025-10-01_15-53-19\Record Node 125\experiment1\recording1
KS4 outputs → C:\Users\ryoi\Documents\SpikeSorting\ks4_outputs
SC2 outputs → C:\Users\ryoi\Documents\SpikeSorting\sc2_outputs
Test duration: 60


In [3]:
#%% 2) Load Open Ephys recording (pick stream safely)
import spikeinterface.extractors as se

# If you didn't set STREAM_NAME above, default to the neural stream (not ADC)
if STREAM_NAME is None:
    STREAM_NAME = "Record Node 125#Acquisition_Board-100.Rhythm Data"

print("Using stream:", STREAM_NAME)

# Load the selected stream
recording = se.read_openephys(DATA_PATH, stream_name=STREAM_NAME)
print(recording)
print("Segments:", recording.get_num_segments(),
      "| Fs:", recording.get_sampling_frequency(),
      "| Ch:", recording.get_num_channels())

# Kilosort4 requires single segment
if recording.get_num_segments() > 1:
    recording = recording.select_segments([0])
    print("Selected segment 0 for single-segment sorting.")

# Optional quick slice for test runs
if TEST_SECONDS is not None:
    fs = recording.get_sampling_frequency()
    recording = recording.frame_slice(0, int(fs * TEST_SECONDS))
    print(f"Recording sliced to first {TEST_SECONDS} s for quick run.")


Using stream: Record Node 125#Acquisition_Board-100.Rhythm Data
OpenEphysBinaryRecordingExtractor: 64 channels - 30.0kHz - 1 segments - 105,877,760 samples 
                                   3,529.26s (58.82 minutes) - int16 dtype - 12.62 GiB
Segments: 1 | Fs: 30000.0 | Ch: 64
Recording sliced to first 60 s for quick run.


In [4]:
#%% 3) Build tetrode groups from real device IDs (auto or manual)
DEV_IDS = list(recording.channel_ids)
print("First 24 device IDs:", DEV_IDS[:24])
print("Total channels:", len(DEV_IDS))

# --- Auto-generate tetrode groups of 4 channels each ---
if len(DEV_IDS) % 4 != 0:
    raise ValueError("Channel count not divisible by 4; define custom groups to match your wiring.")

groups = [DEV_IDS[i:i + 4] for i in range(0, len(DEV_IDS), 4)]

# --- Manual override example (if you want custom wiring) ---
# groups = [
#     ["CH0", "CH1", "CH2", "CH3"],
#     ["CH4", "CH5", "CH6", "CH7"],
#     # ...
# ]

# --- Remove known bad channels (if any) ---
if BAD:
    groups = [[ch for ch in g if ch not in BAD] for g in groups]
    groups = [g for g in groups if len(g) > 0]
    keep = [ch for g in groups for ch in g]
    recording = recording.channel_slice(keep_channel_ids=keep)
    DEV_IDS = list(recording.channel_ids)
    print("Sliced out bad channels. New channel count:", len(DEV_IDS))

print(f"Tetrodes: {len(groups)} | First group: {groups[0]}")


First 24 device IDs: [np.str_('CH40'), np.str_('CH38'), np.str_('CH36'), np.str_('CH34'), np.str_('CH48'), np.str_('CH46'), np.str_('CH44'), np.str_('CH42'), np.str_('CH56'), np.str_('CH54'), np.str_('CH52'), np.str_('CH50'), np.str_('CH58'), np.str_('CH64'), np.str_('CH62'), np.str_('CH60'), np.str_('CH63'), np.str_('CH61'), np.str_('CH59'), np.str_('CH57'), np.str_('CH55'), np.str_('CH53'), np.str_('CH51'), np.str_('CH49')]
Total channels: 64
Tetrodes: 16 | First group: [np.str_('CH40'), np.str_('CH38'), np.str_('CH36'), np.str_('CH34')]


In [5]:
#%% 4) (Optional) Attach 2×2 tetrode geometry for nicer Phy plots
if ATTACH_GEOMETRY:
    from probeinterface import Probe
    import numpy as np

    pitch = 20.0            # µm inside tetrode (2x2)
    dx, dy = 150.0, 150.0   # µm between tetrodes

    # Map channel-id -> row index in recording
    idx_map = {ch: i for i, ch in enumerate(recording.channel_ids)}
    pos = np.zeros((len(idx_map), 2), dtype=float)

    # Place each tetrode as a little square on a grid
    for t, g in enumerate(groups):
        base_xy = np.array([[0, 0], [pitch, 0], [0, pitch], [pitch, pitch]], dtype=float)[:len(g)]
        row, col = divmod(t, TETRODES_PER_ROW)
        offset = np.array([col * dx, row * dy], dtype=float)
        for j, ch in enumerate(g):
            pos[idx_map[ch]] = base_xy[j] + offset

    # Build probe and (IMPORTANT) set device_channel_indices
    pr = Probe(ndim=2)
    pr.set_contacts(positions=pos, shapes='circle', shape_params={'radius': 7})
    pr.set_device_channel_indices(np.arange(pos.shape[0], dtype=int))  # 0..N-1 in recording order

    recording = recording.set_probe(pr)
    print("Geometry attached. Locations shape:", recording.get_channel_locations().shape)
else:
    print("Geometry attachment skipped (ATTACH_GEOMETRY=False)")


Geometry attached. Locations shape: (64, 2)


In [6]:
#%% Preprocess + cache with timestamped folders (avoids Windows locks)
import spikeinterface.preprocessing as spre
from pathlib import Path
from time import strftime

# Bandpass shared
rec_bp = spre.bandpass_filter(recording, freq_min=300, freq_max=6000)

# KS4 branch: per-tetrode CAR
rec_ks4 = spre.common_reference(rec_bp, reference="global", operator="median", groups=groups)

# SC2 branch: no CAR
rec_sc2 = rec_bp

# Timestamped cache paths (no deletion needed)
ts = strftime("%Y%m%d_%H%M%S")
ks4_cache = KS4_OUT / f"cached_ks4_{ts}"
sc2_cache = SC2_OUT / f"cached_sc2_{ts}"

rec_ks4_cached = rec_ks4.save(folder=ks4_cache, format="binary", dtype="float32",
                              chunk_duration="1s")
rec_sc2_cached = rec_sc2.save(folder=sc2_cache, format="binary", dtype="float32",
                              chunk_duration="1s")

print("Cached KS4 →", ks4_cache)
print("Cached SC2 →", sc2_cache)


write_binary_recording 
engine=process - n_jobs=1 - samples_per_chunk=30,000 - chunk_memory=3.66 MiB - total_memory=3.66 MiB - chunk_duration=1.00s


write_binary_recording (no parallelization):   0%|          | 0/60 [00:00<?, ?it/s]

write_binary_recording 
engine=process - n_jobs=1 - samples_per_chunk=30,000 - chunk_memory=3.66 MiB - total_memory=3.66 MiB - chunk_duration=1.00s


write_binary_recording (no parallelization):   0%|          | 0/60 [00:00<?, ?it/s]

Cached KS4 → C:\Users\ryoi\Documents\SpikeSorting\ks4_outputs\cached_ks4_20251024_115046
Cached SC2 → C:\Users\ryoi\Documents\SpikeSorting\sc2_outputs\cached_sc2_20251024_115046


In [8]:
# ---- Utilities: safe folder delete on Windows ----
from pathlib import Path
import shutil, time

def safe_rmtree(path: Path, retries: int = 5, wait_s: float = 0.6):
    """Delete a folder with a few retries to dodge Windows file locks."""
    path = Path(path)
    for _ in range(retries):
        try:
            if path.exists():
                shutil.rmtree(path)
            return
        except PermissionError:
            time.sleep(wait_s)
    # last try—raise if still locked
    if path.exists():
        shutil.rmtree(path)


In [9]:
#%% 7A) Run Kilosort4 (if enabled)
from time import strftime
import spikeinterface.sorters as ss

if RUN_KS4:
    tag = strftime("%Y%m%d_%H%M%S")
    ks4_params = ss.Kilosort4Sorter.default_params()
    ks4_params.update({
        "torch_device": KS4_TORCH_DEVICE,  # "cpu" or "cuda"
        "do_CAR": False,                   # we already did CAR in preprocessing
        "progress_bar": True,
    })

    ks4_out = KS4_OUT / f"ks4_run_{tag}"
    safe_rmtree(ks4_out)  # avoids folder lock errors if rerunning

    sorting_ks4 = ss.run_sorter(
        "kilosort4",
        rec_ks4_cached,
        folder=ks4_out,
        verbose=True,
        remove_existing_folder=True,
        **ks4_params,
    )
    print("KS4 units:", sorting_ks4.get_num_units(), "| out →", ks4_out)
else:
    sorting_ks4 = None
    print("KS4 skipped (RUN_KS4=False)")


kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 1800000
kilosort.run_kilosort: N seconds: 60.0
kilosort.run_kilosort: N batches: 30


Skipping common average reference.


kilosort.run_kilosort: Preprocessing filters computed in 0.27s; total 0.27s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:    40.00 %
kilosort.run_kilosort: Mem used:     72.00 %     |      11.13 GB
kilosort.run_kilosort: Mem avail:     4.33 / 15.47 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    N/A
kilosort.run_kilosort: GPU memory:   N/A
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing drift correction.
kilosort.run_kilosort: ----------------------------------------
kilosort.spikedetect: Re-computing universal templates from data.
C:\Users\ryoi\AppData\Local\anaconda3\envs\spikeinterface\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and 

kilosort4 run time 473.75s
KS4 units: 165 | out → C:\Users\ryoi\Documents\SpikeSorting\ks4_outputs\ks4_run_20251024_115255


In [ ]:
#%% Run SpyKING CIRCUS 2
import spikeinterface.sorters as ss
from pathlib import Path
from time import strftime
import shutil, time

# Helper: safe folder delete (avoids Windows file locks)
def safe_rmtree(path: Path, retries: int = 3, wait_s: float = 0.5):
    for _ in range(retries):
        try:
            if path.exists():
                shutil.rmtree(path)
            return
        except PermissionError:
            time.sleep(wait_s)
    if path.exists():
        shutil.rmtree(path)

# Sanity checks
assert 'rec_sc2_cached' in globals(), "rec_sc2_cached not found. Run the caching cell first."
assert 'SC2_OUT' in globals(), "SC2_OUT not defined. Define your output directory first."

# Default parameters (tweak only keys that exist)
sc2_params = ss.Spykingcircus2Sorter.default_params()
# Example optional tweaks (uncomment if you want):
# sc2_params['job_kwargs'] = {'n_jobs': 4, 'chunk_duration': '2s'}  # parallelism for SC2 internals
# sc2_params['general']['debug'] = True  # if you want extra logs

tag = strftime("%Y%m%d_%H%M%S")
sc2_out = SC2_OUT / f"sc2_run_{tag}"
safe_rmtree(sc2_out)

sorting_sc2 = ss.run_sorter(
    "spykingcircus2",
    rec_sc2_cached,           # ← cached bandpass-only recording (no CAR)
    folder=sc2_out,           # (SI 0.103 uses 'folder=')
    verbose=True,
    remove_existing_folder=True,
    **sc2_params,
)

print("SC2 units:", sorting_sc2.get_num_units(), "| out →", sc2_out)


In [ ]:
#%% 8) Create analyzers (only for the sorters that ran)
from spikeinterface.core import create_sorting_analyzer
from time import strftime

ts = strftime("%Y%m%d_%H%M%S")

an_ks4, an_sc2 = None, None

if sorting_ks4 is not None:
    an_ks4 = create_sorting_analyzer(
        sorting=sorting_ks4,
        recording=rec_ks4_cached,
        folder=KS4_OUT / f"analyzer_ks4_{ts}",
        overwrite=True,
    )
    print("Created analyzer for KS4")

if sorting_sc2 is not None:
    an_sc2 = create_sorting_analyzer(
        sorting=sorting_sc2,
        recording=rec_sc2_cached,
        folder=SC2_OUT / f"analyzer_sc2_{ts}",
        overwrite=True,
    )
    print("Created analyzer for SC2")

print("✅ Analyzer setup complete.")


In [ ]:
#%% 9A) Analyzer compute sequence (KS4)

if an_ks4 is not None:
    print("Computing KS4 analyzer extensions...")

    # Step 1 — must come first
    an_ks4.compute({"random_spikes": {"max_spikes_per_unit": 500, "seed": 42}}, verbose=True)

    # Step 2 — extract waveforms (use 1.5 ms before, 2.5 ms after)
    an_ks4.compute({
        "waveforms": {
            "ms_before": 1.5,
            "ms_after": 2.5,
            "dtype": "float32",
        }
    }, verbose=True, n_jobs=4, chunk_duration="2s")

    # Step 3 — build templates (average waveform per unit)
    an_ks4.compute("templates", verbose=True)

    # Step 4 — spike amplitudes
    an_ks4.compute("spike_amplitudes", verbose=True)

    # Step 5 — principal components (for clustering visualization in Phy)
    an_ks4.compute({"principal_components": {"n_components": 5}}, verbose=True)

    print("✅ KS4 analyzer fully computed.")
else:
    print("KS4 analyzer skipped.")


In [ ]:
#%% 9B) Analyzer compute sequence (SC2)

if an_sc2 is not None:
    print("Computing SC2 analyzer extensions...")
    an_sc2.compute({"random_spikes": {"max_spikes_per_unit": 500, "seed": 42}}, verbose=True)
    an_sc2.compute({
        "waveforms": {"ms_before": 1.5, "ms_after": 2.5, "dtype": "float32"}
    }, verbose=True, n_jobs=4, chunk_duration="2s")
    an_sc2.compute("templates", verbose=True)
    an_sc2.compute("spike_amplitudes", verbose=True)
    an_sc2.compute({"principal_components": {"n_components": 5}}, verbose=True)
    print("✅ SC2 analyzer fully computed.")
else:
    print("SC2 analyzer skipped.")


In [ ]:
#%% 10) Export to Phy

from spikeinterface.exporters import export_to_phy
from time import strftime

ts = strftime("%Y%m%d_%H%M%S")

if an_ks4 is not None:
    phy_ks4 = KS4_OUT / f"phy_ks4_{ts}"
    export_to_phy(an_ks4, output_folder=phy_ks4, remove_if_exists=True)
    print("✅ Exported Phy (KS4) →", phy_ks4)

if an_sc2 is not None:
    phy_sc2 = SC2_OUT / f"phy_sc2_{ts}"
    export_to_phy(an_sc2, output_folder=phy_sc2, remove_if_exists=True)
    print("✅ Exported Phy (SC2) →", phy_sc2)

print("🎉 All done — ready for Phy curation.")
